In [583]:
train_data = [
    ['Green', 3, 'Mango'],
    ['Yellow', 3, 'Mango'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [584]:
header = ['Color', 'Diameter', 'Label']

In [585]:
# returns the unique values
def unique_values(rows, col):
    return set([row[col] for row in rows]) # set returns only unique values in a given list

In [586]:
unique_values(train_data, 0) # unique colors

{'Green', 'Red', 'Yellow'}

In [587]:
unique_values(train_data, 1) # unique diameters

{1, 3}

In [588]:
unique_values(train_data, 2) # unique label

{'Grape', 'Lemon', 'Mango'}

In [589]:
# counts the number of each type of example in a dataset
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label in counts:
            counts[label] += 1 
        else:
            counts[label] = 1
    return counts

In [590]:
class_counts(train_data)

{'Mango': 2, 'Grape': 2, 'Lemon': 1}

In [591]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [592]:
is_numeric(3)

True

In [593]:
is_numeric('cute')

False

In [594]:
# printing our question
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = '=='
        if is_numeric(self.value):
            condition = '>='
        return 'is %s %s %s?' % (header[self.column], condition, str(self.value)) 

In [595]:
Question(1, 3)

is Diameter >= 3?

In [596]:
Question(0, 'Green')

is Color == Green?

In [597]:
example = train_data[0]
q = Question(0, 'Green')
q.match(example)

True

In [598]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows # returns a list with 2 smaller lists 

In [599]:
true_rows, false_rows = partition(train_data, Question(0, 'Red')) # unpacking our bigger data
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [600]:
false_rows

[['Green', 3, 'Mango'], ['Yellow', 3, 'Mango'], ['Yellow', 3, 'Lemon']]

In [601]:
def gini(rows): # calculate the gini impurity 
    counts = class_counts(rows)
    impurity = 1
    for label in counts:
        prob_of_label = counts[label] / float(len(rows))
        impurity -= prob_of_label**2
    return impurity

In [602]:
no_mixing = [['Apples'], ['Apples']]
class_counts(no_mixing)

{'Apples': 2}

In [603]:
gini(no_mixing)

0.0

In [604]:
some_mixing = [['Apple'], ['Orange']]
class_counts(some_mixing)

{'Apple': 1, 'Orange': 1}

In [605]:
gini(some_mixing)

0.5

In [606]:
lots_of_mixing = [['Apple'],
                  ['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)

0.7777777777777777

In [607]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [608]:
current_uncertainty = gini(train_data) # impurity
current_uncertainty

0.6399999999999999

In [609]:
true_rows, false_rows = partition(train_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

In [610]:
true_rows, false_rows = partition(train_data, Question(0, 'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

Why does Red give us more info than Green? We can find this by looking at their respective true_rows and false_rows

In [611]:
true_rows, false_rows = partition(train_data, Question(0, 'Green'))
true_rows, false_rows

([['Green', 3, 'Mango']],
 [['Yellow', 3, 'Mango'],
  ['Red', 1, 'Grape'],
  ['Red', 1, 'Grape'],
  ['Yellow', 3, 'Lemon']])

In [612]:
true_rows, false_rows = partition(train_data, Question(0, 'Red'))
true_rows, false_rows

([['Red', 1, 'Grape'], ['Red', 1, 'Grape']],
 [['Green', 3, 'Mango'], ['Yellow', 3, 'Mango'], ['Yellow', 3, 'Lemon']])

You can see the true_rows of Green is just 1 as compared to 2 of Red, while the false_rows of Green are 3 labels(2 labels) as compared to 2(1 impurity) of Red.

In [613]:
# Find the best question to ask by iterating over every feature/value and calculating the information gain
def find_best_split(rows):
    best_gain = 0 # keeps track of best info gained
    best_question = None # keeps track of the best question to ask
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1# no of columns

    for col in range(n_features):
        values = set([row[col] for row in rows])  # unique values in the column
        for val in values:  # for each value
            question = Question(col, val)
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question


In [614]:
best_gain, best_question = find_best_split(train_data)
best_gain, best_question

(0.37333333333333324, is Diameter >= 3?)

In [615]:
class Leaf:
    """A Leaf node classifies data.
    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [616]:
class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    """
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [617]:
def build_tree(rows):
    """Builds the tree.
    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)
    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)
    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)
    # Recursively build the true branch.
    true_branch = build_tree(true_rows)
    # Recursively build the false branch.
    false_branch = build_tree(false_rows)
    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [618]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    # Print the question at this node
    print (spacing + str(node.question))
    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [619]:
my_tree = build_tree(train_data)
print_tree(my_tree)

is Diameter >= 3?
--> True:
  is Color == Yellow?
  --> True:
    Predict {'Mango': 1, 'Lemon': 1}
  --> False:
    Predict {'Mango': 1}
--> False:
  Predict {'Grape': 2}


In [620]:

def classify(row, node):
    """See the 'rules of recursion' above."""
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [621]:
classify(train_data[0], my_tree)

{'Mango': 1}

In [622]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for label in counts.keys():
        probs[label] = str(int(counts[label] / total * 100)) + "%"
    return probs

In [623]:
print_leaf(classify(train_data[0], my_tree))

{'Mango': '100%'}

In [624]:
print_leaf(classify(train_data[1], my_tree))

{'Mango': '50%', 'Lemon': '50%'}

In [625]:
test_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [626]:

for row in test_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Mango': '100%'}
Actual: Apple. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}
